<a href="https://colab.research.google.com/github/reitezuz/18NES1-2025-/blob/main/week11/kmeans_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# K-means clustering
- basic examples and visualizations

## Datasets

### Countries:

In [ ]:
import requests
import pandas as pd
from sklearn.preprocessing import StandardScaler

def prepare_country_data_from_url(url, local_filename='countries.csv'):
    """
    Download CSV from URL, save locally and prepare data for clustering.

    Parameters:
    url (str): URL to raw CSV
    local_filename (str): local filename to save

    Returns:
    X_scaled (np.ndarray): normalized features
    names (np.ndarray): country names
    """
    # Download and save
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Failed to download file, status code {response.status_code}")
    with open(local_filename, 'wb') as f:
        f.write(response.content)

    # Load and prepare

    # Parse CSV
    df = pd.read_csv(local_filename, sep=';')

    # Features for clustering
    X = df[['Area (sq. mi.)', 'Population']].values

    # Normalize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Names (for visualization or labeling)
    names = df['Country'].values

    return X_scaled, names

X_countries, names_countries = prepare_country_data_from_url('/master/data/countries.csv')


In [2]:

import pandas as pd

# Pozor na správné nastavení oddělovače:
df = pd.read_csv('countries.csv', sep=';')

print(df.head())

           Country  Population  Area (sq. mi.)
0     Afghanistan     31056997          647500
1         Albania      3581655           28748
2         Algeria     32930091         2381740
3  American Samoa        57794             199
4         Andorra        71201             468
